# Language Detection of Document and Translation

* Author: docai-incubator@google.com


## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the DocAI Incubator Team. No guarantees of performance are implied.

## Purpose and Description

This document guides you to use the translation API for detecting the language of the document and also to translate the document to the desired language.

## Prerequisites

1. Vertex AI Notebook
2. Documents in GCS Folder
3. Output folder to upload translated documents
4. Enable Translate API



## Functions to translate the documents and detect language

### 1. Detecting language from Doc AI parsed json( OCR text)

In [ ]:
!pip install google-cloud-translate

In [1]:
# Run this cell to download utilities module
# !wget https://raw.githubusercontent.com/GoogleCloudPlatform/document-ai-samples/main/incubator-tools/best-practices/utilities/utilities.py

In [ ]:
from utilities import documentai_json_proto_downloader


from google.cloud import translate_v3beta1 
def sample_detect_language(project_id : str ,location : str ,content : str):
    """ This Function is used to detect the language using text
    Args:
        project_id: The GCP project ID.
        location : The location of the project.
        content: The text form of the document.

    Returns:
        The language of the document with the confidence score.
    """
    # Create a client
    client = translate_v3beta1.TranslationServiceClient()

    # Initialize request argument(s)
    parent=f'projects/{project_id}/locations/{location}'
    request = translate_v3beta1.DetectLanguageRequest(
        content=content,
        parent=parent,
    )
    # Make the request
    response = client.detect_language(request=request)

    # Handle the response
    print(response)


#Calling the functions#
project_id = <PROJECT ID>
location = "global" #or us-central1
path = 'gs://xxxx/xxxx/xxxxx.json'

path_in_list =  path.split("/")
bucket = path_in_list[2]
prefix_file_path = "/".join(path_in_list[3:])

json_dict=documentai_json_proto_downloader(bucket, prefix_file_path)
content=json_dict.text
sample_detect_language(project_id,location,content)


#### Output


languages {
  language_code: "de"
  confidence: 1.0
}


### 2. Detecting language and Translating the document (Single document)

In [ ]:
from google.cloud import translate_v3beta1 as translate


def translate_document(
    project_id : str,
    gcs_input_path : str,
    gcs_output_path : str

) -> translate.TranslationServiceClient:
    """Translates a document.

    Args:
        project_id: The GCP project ID.
        file_path: The path to the file to be translated.

    Returns:
        The translated document.
    """

    client = translate.TranslationServiceClient()
    location = "us-central1"
    parent = f"projects/{project_id}/locations/{location}"

    document_input_config = {
        #"content": document_content,
        
        'gcs_source':{'input_uri':gcs_input_path},
       # "mime_type": "application/pdf",
    }
    document_output_config={ 'gcs_destination':{'output_uri_prefix':gcs_output_path}}
    response = client.translate_document(
        request={
            "parent": parent,
           # 'source_language_code':'fr',
            "target_language_code": "en",
            "document_input_config": document_input_config,
            'document_output_config':document_output_config
        }
    )

    print(
        f"Response: Detected Language Code - {response.document_translation.detected_language_code}"
    )

    return response


#Calling Function#
project_id= <PROJECT ID>
gcs_input_path='gs://xxxx/xxxx/xxx/xx.pdf'
gcs_output_path='gs://xxxxx/xxx/'
response=translate_document(project_id,gcs_input_path,gcs_output_path)


#### Output

Response: Detected Language Code - fr

Translated document will also be saved in the gcs output path  and file name (format) saved will be as per [doc](https://cloud.google.com/python/docs/reference/translate/latest/google.cloud.translate_v3beta1.types.DocumentOutputConfig)

### 3. Detecting language and Translating the documents ( Batch process)

In [ ]:
from google.cloud import translate_v3beta1 as translate


def batch_translate_document(
    input_uri: str,
    output_uri: str,
    project_id: str,
    source_language_code: str,
    target_language_codes: ['en'],
    timeout: int = 600,
    
) -> translate.BatchTranslateDocumentResponse:
    """Batch translate documents.

    Args:
        input_uri: Google Cloud Storage location of the input document.
        output_uri: Google Cloud Storage location of the output document.
        project_id: The GCP project ID.
        timeout: The timeout for this request.

    Returns:
        Translated document response
    """
    client = translate.TranslationServiceClient()

    # The ``global`` location is not supported for batch translation
    location = "us-central1"

    # Google Cloud Storage location for the source input. This can be a single file
    # (for example, ``gs://translation-test/input.docx``) or a wildcard
    # (for example, ``gs://translation-test/*``).
    # Supported file types: https://cloud.google.com/translate/docs/supported-formats
    gcs_source = {"input_uri": input_uri}

    batch_document_input_configs = {
        "gcs_source": gcs_source,
    }
    gcs_destination = {"output_uri_prefix": output_uri}
    batch_document_output_config = {"gcs_destination": gcs_destination}
    parent = f"projects/{project_id}/locations/{location}"

    # Supported language codes: https://cloud.google.com/translate/docs/language
    operation = client.batch_translate_document(
        request={
            "parent": parent,
            "source_language_code": source_language_code,
            "target_language_codes": target_language_codes,
            "input_configs": [batch_document_input_configs],
            "output_config": batch_document_output_config,
        }
    )

    print("Waiting for operation to complete...")
    response = operation.result(timeout)

    print(f"Total Pages: {response.total_pages}")

    return operation


# example calling function#
input_path = "gs://xxx/xxxxxx/xxxx/*"
output_path = "gs://xxxx/xxxxxx/"
operation=batch_translate_document(input_uri=input_path,output_uri=output_path,
                                   project_id=project_id,source_language_code="fr-FR",
                                   target_language_codes=["en",'hi'],timeout= 600)

The target language code can be multiple languages and the output translated documents are saved in the output path given and also with a ‘index.csv’ file which has file path and names from source and destination as well .

<img src="./Images/language_detection_output_1.png" width=800 height=400 alt="Language detection bucket Output image">

<img src="./Images/language_detection_output_2.png" width=800 height=400 alt="Language detection CSV output image">

## Supported Languages and File types [Link](https://cloud.google.com/translate/docs/advanced/translate-documents)


<img src="./Images/language_detection_output_3.png" width=800 height=400 alt="Language detection supported language image">

## Supported Languages  [Link](https://cloud.google.com/translate/docs/advanced/discovering-supported-languages-v3)


### 1. Function to get all the supported languages in the project

In [ ]:
from google.cloud import translate


def get_supported_languages(
    project_id: str = "YOUR_PROJECT_ID",
) -> translate.SupportedLanguages:
    """Getting a list of supported language codes.

    Args:
        project_id: The GCP project ID.

    Returns:
        A list of supported language codes.
    """
    client = translate.TranslationServiceClient()

    parent = f"projects/{project_id}"

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    response = client.get_supported_languages(parent=parent)

    # List language codes of supported languages.
    print("Supported Languages:")
    for language in response.languages:
        print(f"Language Code: {language.language_code}")

    return response

### 2. Function to get all the supported languages with target language

In [4]:
from google.cloud import translate


def get_supported_languages_with_target(
    project_id: str = "rand-automl-project",#"YOUR_PROJECT_ID",
) -> translate.SupportedLanguages:
    """Listing supported languages with target language name.

    Args:
        project_id: Your Google Cloud project ID.

    Returns:
        Supported languages.
    """
    client = translate.TranslationServiceClient()

    parent = f"projects/{project_id}"

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    response = client.get_supported_languages(
        display_language_code="fr", parent=parent  # target language code
    )
    # List language codes of supported languages
    for language in response.languages:
        print(f"Language Code: {language.language_code}")
        print(f"Display Name: {language.display_name}")

    return response
